In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Aug  4 12:43:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

In [4]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [ ]:
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 784))
# X_test = np.reshape(X_test, (-1, 784))

In [ ]:
# cifar10 = tf.keras.datasets.cifar10
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 3072))
# X_test = np.reshape(X_test, (-1, 3072))

In [31]:
from sklearn.model_selection import train_test_split

glass = pd.read_csv('glass.data', header=None)
y = glass.pop(10)
X = glass
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = X_train.values.astype(dtype)
X_test = X_test.values.astype(dtype)
y_train = np.reshape(y_train.values, (-1, 1)).astype(dtype)
y_test = np.reshape(y_test.values, (-1, 1)).astype(dtype)

In [18]:
from sklearn.model_selection import train_test_split

vehicle = pd.read_csv('vehicle.txt', header=None, delim_whitespace=True)

vehicle[18] = pd.Categorical(vehicle[18])
vehicle[18] = vehicle[18].cat.codes

y = vehicle.pop(18)
X = vehicle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = X_train.values.astype(dtype)
X_test = X_test.values.astype(dtype)
y_train = np.reshape(y_train.values, (-1, 1)).astype(dtype)
y_test = np.reshape(y_test.values, (-1, 1)).astype(dtype)

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [33]:
class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # The scaling vector could be [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        # and the resulting weighted values could be
        #
        # [[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_vector = tf.cumsum(tf.constant(self.regularization_penalty, shape=(x.shape[-1],), dtype=dtype), axis=0)
        weighted_values = scaling_vector * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class SSLayer(tf.keras.Model):
    def __init__(self, input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer, regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = SSRegularizer(self.regularization_penalty, self.regularization_method)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = SSLayer(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy


class SSModel(tf.keras.Model):
    def __init__(self, layer_sizes, activation=None, regularization_penalty=0.01, regularization_method='weighted_l1', kernel_initializer='glorot_uniform', bias_initializer='zeros'):
        super().__init__()
        
        self.sslayers = list()
        for l in range(len(layer_sizes) - 1):
            input_units = layer_sizes[l]
            units = layer_sizes[l + 1]
            if l < len(layer_sizes) - 2:
                layer = SSLayer(input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer)
            else:  # Last layer
                layer = SSLayer(input_units, units, 'softmax', 0., regularization_method, kernel_initializer, bias_initializer)
            self.sslayers.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.sslayers:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.sslayers)):
            layer = self.sslayers[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.sslayers) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def print_neurons(self):
        for layer in self.sslayers[:-1]:
            print(get_param_string(layer.W, layer.b))
    
    def remove_regularization(self):
        for l in range(len(self.sslayers)):
            self.sslayers[l] = self.sslayers[l].copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.sslayers[0].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for l in range(0, len(self.sslayers) - 1):  # Every layer except of the last is regularized
            self.sslayers[l].regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * 0.2))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)

In [34]:
def get_param_string(weights, bias):
    param_string = ""
    weights_with_bias = tf.concat([weights, tf.reshape(bias, (1, -1))], axis=0)
    max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
    magnitudes = np.floor(np.log10(max_parameters))
    for m in magnitudes:
        if m > 0:
            m = 0
        param_string += str(int(-m))
    return param_string


def print_epoch_statistics(model, x, y, validation_data, print_neurons):
    x_val = validation_data[0]
    y_val = validation_data[1]

    y_pred = model(x)
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, y_pred))
    accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y, y_pred))
    
    y_val_pred = model(x_val)
    val_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_val, y_val_pred))
    val_accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_val, y_val_pred))
    print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
    hidden_layer_sizes = model.get_hidden_layer_sizes()
    print(f"hidden layer sizes: {hidden_layer_sizes}, total neurons: {sum(hidden_layer_sizes)}")
    if print_neurons:
        model.print_neurons()

    return val_loss


def train_model(model, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, regularization_penalty_multiplier=1., print_neurons=False):
    train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    best_val_loss = np.inf
    training_stalled = False
    for epoch in range(epochs):
        print("##########################################################")
        print(f"Epoch {epoch + 1}/{epochs}")

        if epoch < self_scaling_epochs:
            print("Before growing:")

            val_loss = print_epoch_statistics(model, x, y, validation_data, print_neurons)
            if val_loss >= best_val_loss:
                if not training_stalled:
                    penalty = model.get_regularization_penalty() * regularization_penalty_multiplier
                    model.set_regularization_penalty(penalty)
                    training_stalled = True
            else:
                best_val_loss = val_loss
                training_stalled = False

            model.grow(min_new_neurons=min_new_neurons, scaling_factor=pruning_threshold)
            print("After growing:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
        
        if epoch == self_scaling_epochs:
            model.remove_regularization()

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred = model(x_batch, training=True)
                loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                loss_value += sum(model.losses)

            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if epoch < self_scaling_epochs:
            print("Before pruning:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
            model.prune(threshold=pruning_threshold)
            print("After pruning:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
        else:
            print_epoch_statistics(model, x, y, validation_data, print_neurons)

# Small datasets

## Glass dataset

**Here the results of an auto-sizing network are very good compared to a static network, with the accuracy typically improved by several percent:**

In [35]:
epochs = 60
self_scaling_epochs = 40
batch_size = 32
min_new_neurons = 50

In [36]:
%%time

model = SSModel(layer_sizes=[10, 100, 100, 100, 100, 8], activation='selu', regularization_penalty=0.001, 
                regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
            min_new_neurons, validation_data=(X_test_norm, y_test), pruning_threshold=0.01, print_neurons=True)

##########################################################
Epoch 1/60
Before growing:
loss: 2.8872628211975098 - accuracy: 0.06711409240961075 - val_loss: 2.902275562286377 - val_accuracy: 0.0615384615957737 - penalty: 0.001
hidden layer sizes: [100, 100, 100, 100], total neurons: 400
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
After growing:
loss: 2.8873229026794434 - accuracy: 0.06711409240961075 - val_loss: 2.9023067951202393 - val_accuracy: 0.0615384615957737 - penalty: 0.001
hidden layer sizes: [150, 150, 150, 150], total neurons: 600
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [37]:
%%time

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(66, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(13, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(27, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(8, activation='softmax', kernel_initializer='lecun_normal'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), metrics=['accuracy'])

model.fit(X_train_norm, y_train, epochs=60, validation_data=(X_test_norm, y_test))

Epoch 1/60
5/5 [==============================] - 1s 37ms/step - loss: 1.9841 - accuracy: 0.2483 - val_loss: 1.5957 - val_accuracy: 0.4462
Epoch 2/60
5/5 [==============================] - 0s 7ms/step - loss: 1.5255 - accuracy: 0.5503 - val_loss: 1.3579 - val_accuracy: 0.5692
Epoch 3/60
5/5 [==============================] - 0s 7ms/step - loss: 1.2465 - accuracy: 0.6711 - val_loss: 1.1825 - val_accuracy: 0.6462
Epoch 4/60
5/5 [==============================] - 0s 7ms/step - loss: 1.0553 - accuracy: 0.7114 - val_loss: 1.0615 - val_accuracy: 0.6615
Epoch 5/60
5/5 [==============================] - 0s 8ms/step - loss: 0.9209 - accuracy: 0.7383 - val_loss: 0.9656 - val_accuracy: 0.6615
Epoch 6/60
5/5 [==============================] - 0s 11ms/step - loss: 0.8147 - accuracy: 0.7651 - val_loss: 0.9055 - val_accuracy: 0.7077
Epoch 7/60
5/5 [==============================] - 0s 7ms/step - loss: 0.7374 - accuracy: 0.7584 - val_loss: 0.8504 - val_accuracy: 0.7385
Epoch 8/60
5/5 [================

## Vehicle dataset

In [22]:
epochs = 60
self_scaling_epochs = 40
batch_size = 32
min_new_neurons = 50

In [29]:
%%time

model = SSModel(layer_sizes=[18, 100, 100, 100, 100, 4], activation='selu', regularization_penalty=0.0001, 
                regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
            min_new_neurons, validation_data=(X_test_norm, y_test), pruning_threshold=0.01, print_neurons=True)

##########################################################
Epoch 1/60
Before growing:
loss: 1.556831955909729 - accuracy: 0.19763512909412384 - val_loss: 1.6075917482376099 - val_accuracy: 0.13385826349258423 - penalty: 0.0001
hidden layer sizes: [100, 100, 100, 100], total neurons: 400
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
After growing:
loss: 1.5568643808364868 - accuracy: 0.19763512909412384 - val_loss: 1.6076170206069946 - val_accuracy: 0.13385826349258423 - penalty: 0.0001
hidden layer sizes: [150, 150, 150, 150], total neurons: 600
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [30]:
%%time

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(114, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(49, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(72, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(58, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(8, activation='softmax', kernel_initializer='lecun_normal'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), metrics=['accuracy'])

model.fit(X_train_norm, y_train, epochs=60, validation_data=(X_test_norm, y_test))

Epoch 1/60
19/19 [==============================] - 1s 10ms/step - loss: 1.4100 - accuracy: 0.4611 - val_loss: 1.0092 - val_accuracy: 0.5354
Epoch 2/60
19/19 [==============================] - 0s 4ms/step - loss: 0.8228 - accuracy: 0.6301 - val_loss: 0.6787 - val_accuracy: 0.6969
Epoch 3/60
19/19 [==============================] - 0s 4ms/step - loss: 0.6574 - accuracy: 0.6858 - val_loss: 0.6101 - val_accuracy: 0.7441
Epoch 4/60
19/19 [==============================] - 0s 3ms/step - loss: 0.5506 - accuracy: 0.7449 - val_loss: 0.5140 - val_accuracy: 0.7795
Epoch 5/60
19/19 [==============================] - 0s 4ms/step - loss: 0.4898 - accuracy: 0.7432 - val_loss: 0.4934 - val_accuracy: 0.7480
Epoch 6/60
19/19 [==============================] - 0s 3ms/step - loss: 0.5085 - accuracy: 0.7568 - val_loss: 0.5644 - val_accuracy: 0.7323
Epoch 7/60
19/19 [==============================] - 0s 3ms/step - loss: 0.5140 - accuracy: 0.7517 - val_loss: 0.4977 - val_accuracy: 0.6811
Epoch 8/60
19/19 [=